In [ ]:
import numpy as np
from operator import eq, ge, le

In [ ]:
# Argument coefficients
As = np.array([
    [[3, 1, -1, 1],
     [5, 1, 1, -1],
     [1, 0, 0, 0],
     [0, 1, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 0, 1]],
    [[1, -3, -1, -2],
     [1, -1, 1, 0],
     [1, 0, 0, 0],
     [0, 1, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 0, 1]],
    [[1, 1, 0, 2, 1],
     [1, 1, 1, 3, 2],
     [0, 1, 1, 2, 1],
     [1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0],
     [0, 0, 1, 0, 0],
     [0, 0, 0, 1, 0],
     [0, 0, 0, 0, 1]],
    [[1, 1, 2, 0, 0],
     [0, -2, -2, 1, -1],
     [1, -1, 6, 1, 1],
     [1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0],
     [0, 0, 1, 0, 0],
     [0, 0, 0, 1, 0],
     [0, 0, 0, 0, 1]],
    [[1, 1, -1, -10],
     [1, 14, 10, -10],
     [1, 0, 0, 0],
     [0, 1, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 0, 1]],
    [[1, 3, 3, 1],
     [2, 0, 3, -1],
     [1, 0, 0, 0],
     [0, 1, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 0, 1]],
    [[3, 1, 1, 1, -2],
     [6, 1, 2, 3, -4],
     [10, 1, 3, 6, -7],
     [1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0],
     [0, 0, 1, 0, 0],
     [0, 0, 0, 1, 0],
     [0, 0, 0, 0, 1]]
], dtype=object)

# Right-side vectors
Bs = np.array([
      [4, 4, 0, 0, 0, 0],
      [-4, 0, 0, 0, 0, 0],
      [5, 9, 6, 0, 0, 0, 0, 0],
      [4, -6, 12, 0, 0, 0, 0, 0],
      [0, 11, 0, 0, 0, 0],
      [3, 4, 0, 0, 0, 0],
      [10, 20, 30, 0, 0, 0, 0, 0]
], dtype=object)

# Objective function coefficients
Cs = np.array([
      [-6, -1, -4, 5],
      [-1, -2, -3, 1],
      [-1, -2, -1, 3, -1],
      [-1, -1, -1, 1, -1],
      [-1, 4, -3, 10],
      [-1, 5, 1, -1],
      [-1, -1, 1, -1, 2]
], dtype=object)

# Relation signs
Rels = np.array([
        [eq, eq, ge, ge, ge, ge],
        [eq, eq, ge, ge, ge, ge],
        [eq, eq, eq, ge, ge, ge, ge, ge],
        [eq, eq, eq, ge, ge, ge, ge, ge],
        [eq, eq, ge, ge, ge, ge],
        [le, le, ge, ge, ge, ge],
        [eq, eq, eq, ge, ge, ge, ge, ge]
], dtype=object)

In [ ]:
# Convert less or equal expression to equation 
def le_to_eq(ind, row):
    Rels[ind][row] = eq
    for i in range(len(As[ind])):
        As[ind][i].append(0 if i != row else 1)

# Convert max problem to min
def ge_to_le(ind, row):
    Rels[ind][row] = le
    for i in range(len(As[ind][row])):
        As[ind][row][i] *= -1
    Bs[ind][row] *= -1

In [ ]:
M = 1e9
EPS = 1e-12

# Simplex method for artificial basis (min problem only)
def simplex_method(A, b, f):
    f = -np.array(f.copy(), dtype=float)
    A = np.array(A.copy(), dtype=float)
    b = np.array(b.copy(), dtype=float)

    A = np.hstack((A, np.eye(len(b))))
    f = np.hstack((f, np.repeat(-M, len(b))))

    for i in range(len(b)):
        if b[i] < 0:
            A[i] = -A[i]
            b[i] = -b[i]
        
        f += A[i] * M

    col = np.argmax(f)
    basis = np.arange(len(A)) + len(A[0]) - len(b)

    while f[col] > EPS:
        row = -1
        min_b = np.inf

        for i in range(len(A)):
            if A[i, col] > EPS:
                b_res = b[i] / A[i, col]

                if b_res < min_b:
                    row = i
                    min_b = b_res

        res_elm = A[row, col]
        basis[row] = col

        for i in range(len(A)):
            if i != row and A[i, col] != 0:
                k = A[i, col] / res_elm
                A[i] = A[i] - A[row] * k
                b[i] = b[i] - b[row] * k

        k = f[col] / res_elm
        f -= A[row, :] * k
        A[row] = A[row] / res_elm
        b[row] = b[row] / res_elm

        col = np.argmax(f)

    res = [0 for _ in range(len(A[0]) - len(b))]
    for i in range(len(basis)):
        res[basis[i]] = b[i] / A[i, basis[i]]

    return res

In [ ]:
for i in range(len(As)):
    for j in range(len(As[i])):
        if (Rels[i][j] == ge):
            ge_to_le(i, j)
        if (Rels[i][j] == le):
            le_to_eq(i, j)
    
    for j in range(len(As[i])):
        delta = len(As[i]) - len(As[i][j]) + len(Cs[i])
        for k in range(delta):
            As[i][j].append(0)

    for j in range(len(As[i])):
        Cs[i].append(0)

    res = simplex_method(As[i], Bs[i], Cs[i])
    print(f'Answer for system {i + 1}:\n{res}\n')

Answer for system 1:
[0, 4.0, 0.0, 0, 0.0, 4.0, 0.0, 0.0, 0, 0]

Answer for system 2:
[2.0, 2.0, 0, 0, 2.0, 2.0, 0.0, 0.0, 0, 0]

Answer for system 3:
[3.0, 2.0, 4.0, 0, 0, 3.0, 2.0, 4.0, 0.0, 0.0, 0, 0, 0]

Answer for system 4:
[4.0, 0, 0, 1.0, 7.0, 4.0, 0.0, 0.0, 1.0, 7.0, 0, 0, 0]

Answer for system 5:
[0.9999999999999999, 0, 0.9999999999999998, 0, 0.9999999999999998, 0.0, 0.9999999999999998, 0.0, 0, 0]

Answer for system 6:
[0, 0, 0, 3.0, 0, 7.0, 0.0, 0.0, 0.0, 3.0]

Answer for system 7:
[4.440892098500626e-16, 2.2204460492503147e-15, 9.999999999999993, 2.2204460492503147e-15, 0, 0, 2.2204460492503147e-15, 9.999999999999993, 2.2204460492503147e-15, 0.0, 0, 0, 0]

